# Fine-tune Llama3 and open LLMs on AWS Trainium 


Open LLMs like Meta [Llama 3](https://huggingface.co/meta-llama/Meta-Llama-3-70b), Mistral AI [Mistral](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) & [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) models or AI21 [Jamba](https://huggingface.co/ai21labs/Jamba-v0.1) are now OpenAI competitors. However, most of the time you need to fine-tune the model on your data to unlock the full potential of the model. Fine-tuning smaller LLMs, like Mistral became very accessible but still require a lot of computational resources. Thats were AWS Trainium comes into play.


This blog post walks you thorugh how to fine-tune a Llama 3 8B using [Hugging Face Optimum Neuron](https://huggingface.co/docs/optimum-neuron/index) on AWS Trainium. We will use the `NeuronTrainer` to fine-tune the model on a custom dataset. The `NeuronTrainer` is a high-level API that allows you to fine-tune large language models on AWS Trainium with ease. You will learn how to:

1. [Setup AWS environment](#1-setup-aws-environment)
2. [Create and prepare the dataset](#2-create-and-prepare-the-dataset)
3. [Fine-tune Llama3 on AWS Trainium using the `NeuronTrainer`](#3-fine-tune-llama-on-aws-trainium-using-the-neurontrainer)
4. [Evalaute and test fine-tuned model](#4-evalaute-and-test-fine-tuned-llama-model)

## Quick intro: AWS Trainium

[AWS Trainium (Trn1)](https://aws.amazon.com/de/ec2/instance-types/trn1/) is a purpose-built EC2 for deep learning (DL) training workloads. Trainium is the successor of [AWS Inferentia](https://aws.amazon.com/ec2/instance-types/inf1/?nc1=h_ls) focused on high-performance training workloads. Trainium has been optimized for training natural language processing, computer vision, and recommender models used. The accelerator supports a wide range of data types, including FP32, TF32, BF16, FP16, UINT8, and configurable FP8. 

The biggest Trainium instance, the `trn1.32xlarge` comes with over 500GB of memory, making it easy to fine-tune ~10B parameter models on a single instance. Below you will find an overview of the available instance types. More details [here](https://aws.amazon.com/de/ec2/instance-types/trn1/#Product_details):

| instance size | accelerators | accelerator memory | vCPU | CPU Memory | price per hour |
| --- | --- | --- | --- | --- | --- |
| trn1.2xlarge | 1 | 32 | 8 | 32 | \$1.34 |
| trn1.32xlarge | 16 | 512 | 128 | 512 | \$21.50 |
| trn1n.32xlarge (2x bandwidth) | 16 | 512 | 128 | 512 | \$24.78 |

---

*Note: This tutorial was created on a trn1.32xlarge AWS EC2 Instance.* 


## 1. Setup AWS environment

In this example, we will use the `trn1.32xlarge` instance on AWS with 16 Accelerator, including 32 Neuron Cores and the [Hugging Face Neuron Deep Learning AMI](https://aws.amazon.com/marketplace/pp/prodview-gr3e6yiscria2). The Hugging Face AMI comes with all important libraries, like Transformers, Datasets, Optimum and Neuron packages pre-installed this makes it super easy to get started, since there is no need for environment management.

This blog post doesn’t cover how to create the instance in detail. You can check out my previous blog about [“Setting up AWS Trainium for Hugging Face Transformers”](https://www.philschmid.de/setup-aws-trainium), which includes a step-by-step guide on setting up the environment. 

Once the instance is up and running, we can ssh into it. But instead of developing inside a terminal we want to use a `Jupyter` environment, which we can use for preparing our dataset and launching the training. For this, we need to add a port for forwarding in the `ssh` command, which will tunnel our localhost traffic to the Trainium instance.

```bash
PUBLIC_DNS="" # IP address, e.g. ec2-3-80-....
KEY_PATH="" # local path to key, e.g. ssh/trn.pem

ssh -L 8080:localhost:8080 -i ${KEY_NAME}.pem ubuntu@$PUBLIC_DNS
```

Next we can change our directory to `TODO/CHANGE` and launch the `jupyter` environment.


```bash
# change directory
cd optimum-neuron/notebooks/text-generation
# launch jupyter
python -m notebook --allow-root --port=8080
```

You should see a familiar **`jupyter`** output with a URL to the notebook.

**`http://localhost:8080/?token=8c1739aff1755bd7958c4cfccc8d08cb5da5234f61f129a9`**

We can click on it, and a **`jupyter`** environment opens in our local browser. Open the notebook **`llama3-8b-fine-tuning.ipynb`** and lets get started.

_Note: We are going to use the Jupyter environment only for preparing the dataset and then `torchrun` for launching our training script for distributed training._

First we need to login into our Hugging Face account, which has access to the model, to use your token for accessing the gated repository. We can do this by running the following command:

_Note: We also provide an ungated checkpoint._

In [ ]:
!huggingface-cli login --token YOUR_TOKEN

## 2. Create and prepare the dataset

After our environment is set up, we can start creating and preparing our dataset. A fine-tuning dataset should have a diverse set of demonstrations of the task you want to solve. If you want to learn more about how to create a dataset, take a look at the [How to Fine-Tune LLMs in 2024 with Hugging Face](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl#3-create-and-prepare-the-dataset).

We will use the [HuggingFaceH4/no_robots](https://huggingface.co/datasets/HuggingFaceH4/no_robots) dataset a high-quality dataset of 10,000 instructions and demonstrations created by skilled human annotators. This data can be used for supervised fine-tuning (SFT) to make language models follow instructions better. No Robots was modelled after the instruction dataset described in OpenAI's [InstructGPT paper](https://huggingface.co/papers/2203.02155), and is comprised mostly of single-turn instructions.

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

The [no_robots](https://huggingface.co/datasets/HuggingFaceH4/no_robots) dataset has 10,000 split into 9,500 training and 500 test examples. Some samples are not including a `system` message. We will load the dataset with the `datasets` library, add a missing `system` message and save them to separate json files.

In [1]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are Llama, an AI assistant created by Philipp to be helpful and honest. Your knowledge spans a wide range of topics, allowing you to engage in substantive conversations and provide analysis on complex subjects."""

def create_conversation(sample):
    if sample["messages"][0]["role"] == "system":
        return sample
    else:
      sample["messages"] = [{"role": "system", "content": system_message}] + sample["messages"]
      return sample

# Load dataset from the hub
dataset = load_dataset("HuggingFaceH4/no_robots")

# Add system message to each conversation
columns_to_remove = list(dataset["train"].features)
columns_to_remove.remove("messages")
dataset = dataset.map(create_conversation, remove_columns=columns_to_remove,batched=False)

# Filter out conversations which are corrupted with wrong turns, keep which have even number of turns after adding system message
dataset["train"] = dataset["train"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)
dataset["test"] = dataset["test"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)

# save datasets to disk 
dataset["train"].to_json("train_dataset.json", orient="records", force_ascii=False)
dataset["test"].to_json("test_dataset.json", orient="records", force_ascii=False)

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

784047

## 3. Fine-tune Llama on AWS Trainium using the `NeuronTrainer`

We are now ready to fine-tune our LLM with [NeuronTrainer](https://huggingface.co/docs/optimum-neuron/package_reference/trainer) a 1-to-1 replacement for the Hugging Face `Trainer` but for AWS Trainium instances.

Every AWS Trainium instances comes with > 1 accelerators. This means we will always use distributed training. The `NeuronTrainer` comes with different distributed training strategies including: 
* [ZeRO-1](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/frameworks/torch/torch-neuronx/tutorials/training/zero1_gpt2.html): shards the optimizer state over multiple devices.
* [Tensor Parallelism](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/neuronx-distributed/tensor_parallelism_overview.html): shards the model parameters along a given dimension on multiple devices, defined with `tensor_parallel_size`
* [Sequence parallelism](https://arxiv.org/pdf/2205.05198.pdf) shards the activations on the sequence axis outside of the tensor parallel regions. It is useful because it saves memory by sharding the activations.
* [Pipeline Parallelism](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/neuronx-distributed/pipeline_parallelism_overview.html): _coming soon_


We prepared a script [run_sft.py](../scripts/run_sft.py) which will load the dataset from disk (json), prepare the model, tokenizer and making it straightfoward to supervise fine-tune open LLMs supporting:
* Config loading from YAML files
* Dataset formatting, including conversational and instruction format (OpenAI conmpatible)
* Packing datasets for more efficient training
* PEFT (parameter-efficient fine-tuning) support including LoRA (_soon_)

When training models on AWS Accelerators like AWS Trainium, we must first compile our model to enable execution on the specialized hardware. During compilation, the model's computational graph is optimized and translated into instructions specifically tailored for Trainium's NeuronCores, ensuring efficient utilization of the accelerator's capabilities.

Model compilation is done using the `neuron_parallel_compile` with your training script, model and hyperparameters you plan to use during training, except that it only needs to be run on a few steps, e.g. `10`. The `run_sft.py` scripts support loading config files, which makes it easy to overwrite the default parameters.

First lets create our config file `config.yaml`:

In [1]:
%%writefile llama_3_8b.yaml
# script parameters
model_id: "meta-llama/Meta-Llama-3-8b" # Hugging Face model id
dataset_path: "notebooks/train_dataset.json"     # path to dataset
max_seq_len: 2048                      # max sequence length for model and packing of the dataset
# training parameters
output_dir: "./llama3_trn"             # output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 5.0e-5                  # learning rate 5e-5
lr_scheduler_type: "cosine"            # learning rate scheduler
num_train_epochs: 3                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 16        # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
bf16: true                             # use bfloat16 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# distributed parameters
tensor_parallel_size: 8                # number of tensor parallel groups
zero_1: true                           # use zero stage 1

Writing llama_3_8b.yaml


After we created our config file, we can pre-compile our model using the `neuron_parallel_compile` command. We will use the `run_sft.py` script to compile the model. 

In [ ]:
# precompilation command 
!MALLOC_ARENA_MAX=64 neuron_parallel_compile torchrun --nproc_per_node=32 ../scripts/run_sft.py --config config.yaml --max_steps 10

_Note: Compiling without a cache can take ~40 minutes. It will also create dummy files in the `llama3_trn` during compilation you we have to remove them afterwards. We also need to add `MALLOC_ARENA_MAX=64` to limit the CPU allocation to avoid potential crashes, don't remove it for now._ 

In [ ]:
# remove dummy artifacts which are created by the precompilation command
!rm -rf "llama3_trn"

After the compilation is done we can start our training with a similar command, we just need to remove the `neuron_parallel_compile` and `max_steps` and can launch the training with the following command.

In [ ]:
!MALLOC_ARENA_MAX=64 torchrun --nproc_per_node=32 ../scripts/run_sft.py --config config.yaml

The training for 3 epochs on dolly (15k samples) took 43:24 minutes where the raw training time was only 31:46 minutes. This leads to a cost of ~$15.5 for the e2e training on the trn1.32xlarge instance. Not Bad! 

But before we can share and test our model we need to consolidate our model. Since we used Tensor Parallelism during training, we need to consolidate the model weights before we can use it. Tensor Parallelism shards the model weights accross different workers, only sharded checkpoints will be saved during training.

The Optimum CLI provides a way of doing that very easily via the `optimum neuron consolidate`` command:

In [ ]:
!optimum-cli neuron consolidate llama3_trn/tensor_parallel_shards llama3_trn

Lets remove our "sharded" checkpoints as we have consolidated them already to safetensors.

In [ ]:
!rm -rf llama3_trn/tensor_parallel_shards

## 4. Compile and test Llama 3 for inference

Similar to training to be able to run inferece on AWS Trainium or AWS Inferentia2 we need to compile our model for the correct use. We will use our Trainium instance for the inference test, but we recommend customer to switch to Inferentia2 for inference. 

Optimum Neuron implements similar to Transformers AutoModel classes for easy inference use. We will use  the `NeuronModelForCausalLM` class to load our vanilla transformers checkpoint and convert it to neuron. 

In [ ]:
from optimum.neuron import NeuronModelForCausalLM
from transformers import AutoTokenizer

compiler_args = {"num_cores": 2, "auto_cast_type": 'fp16'}
input_shapes = {"batch_size": 1, "sequence_length": 2048}

tokenizer = AutoTokenizer.from_pretrained("dolly_llama")
model = NeuronModelForCausalLM.from_pretrained(
        "llama3_trn",
        export=True,
        **compiler_args,
        **input_shapes)


_Note: Inference compilation can take ~25minutes. Luckily, you need to only run this onces. Since you can save the model afterwards. If you are going to run on Inferentia2 you need to recompile again. The compilation is parameter and hardware specific._

In [ ]:
# COMMENT IN if you want to save the compiled model
# model.save_pretrained("compiled_dolly_llama")

We can now test inference, but have to make sure we format our input to our prompt format we used for fine-tuning. Therefore we created a helper method, which accepts a `dict` with our `instruction` and optionally a `context`. 

In [ ]:
def generate(messages): 
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(**inputs,
                         max_new_tokens=512,
                         do_sample=True,
                         temperature=0.9,
                         top_k=50,
                         top_p=0.9
                         )
    return tokenizer.decode(outputs[0][inputs:], skip_special_tokens=False)

Lets test inference. First we test without a context.

_Note: Inference is not expected to be super fast on AWS Trainium using 2 cores. For Inference we recommend using Inferentia2._

In [ ]:
messages = [
  { "role": "system", "content": "You are an expert on AWS." },
  { "role": "user", "content": "Can you tell me something about AWS?" },
]
res = generate(messages)

print(res)

> AWS stands for Amazon Web Services. AWS is a suite of remote computing services offered by Amazon. The most widely used of these include Amazon Elastic Compute Cloud (Amazon EC2), which provides resizable compute capacity in the cloud; Amazon Simple Storage Service (Amazon S3), which is an object storage service; and Amazon Elastic Block Store (Amazon EBS), which is designed to provide high performance, durable block storage volumes for use with AWS instances. AWS also provides other services, such as AWS Identity and Access Management (IAM), a service that enables organizations to control access to their AWS resources, and AWS Key Management Service (AWS KMS), which helps customers create and control the use of encryption keys.</s>